In [12]:
import open3d as o3d
import numpy as np

# Assuming 'blocks' and 'neighbors' are available and each block has XYZ information
def visualize_block_and_neighbors(block, neighbors):
    # Create Open3D PointCloud object for the block
    point_cloud_block = o3d.geometry.PointCloud()
    
    # Extract XYZ coordinates for the block
    xyz_block = block[:, :3]

    # Set points for the block point cloud
    point_cloud_block.points = o3d.utility.Vector3dVector(xyz_block)

    # Set block color to red (same color for all points)
    point_cloud_block.paint_uniform_color([1.0, 0.0, 0.0])

    # Create Open3D PointCloud object for the neighbors
    point_cloud_neighbors = o3d.geometry.PointCloud()

    # Extract XYZ coordinates for the neighbors (using neighbor indices to access the block points)
    if len(neighbors) > 0:
        xyz_neighbors = block[neighbors.flatten(), :3]
        point_cloud_neighbors.points = o3d.utility.Vector3dVector(xyz_neighbors)

        # Set neighbors color to blue (same color for all points)
        point_cloud_neighbors.paint_uniform_color([0.0, 0.0, 1.0])
        
        print(f"Number of neighbor points: {len(xyz_neighbors)}")
    else:
        print("No neighbor points found.")

    # Visualize both the block and its neighbors
    o3d.visualization.draw_geometries([point_cloud_block, point_cloud_neighbors])

# Visualize the first block of points and its neighbors
visualize_block_and_neighbors(blocks[1], neighbors[1])



Number of neighbor points: 10240


In [5]:
import open3d as o3d
import numpy as np

# Assuming 'blocks' is available and each block has XYZ and potentially RGB information
def visualize_all_blocks(blocks):
    for i, block in enumerate(blocks):
        print(f"Visualizing block {i + 1}/{len(blocks)}")
        visualize_block(block)

def visualize_block(block):
    # Create Open3D PointCloud object
    point_cloud = o3d.geometry.PointCloud()

    # Extract XYZ coordinates
    xyz = block[:, :3]

    # Set points for the point cloud
    point_cloud.points = o3d.utility.Vector3dVector(xyz)

    # Check if RGB information is available
    if block.shape[1] >= 6:
        rgb = block[:, 3:6] / 255.0  # Assuming RGB values are between 0-255, normalize to 0-1
        point_cloud.colors = o3d.utility.Vector3dVector(rgb)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])

# Visualize all blocks of points
visualize_all_blocks(blocks)


Visualizing block 1/580
Visualizing block 2/580
Visualizing block 3/580
Visualizing block 4/580
Visualizing block 5/580
Visualizing block 6/580
Visualizing block 7/580
Visualizing block 8/580
Visualizing block 9/580
Visualizing block 10/580
Visualizing block 11/580
Visualizing block 12/580
Visualizing block 13/580
Visualizing block 14/580
Visualizing block 15/580
Visualizing block 16/580
Visualizing block 17/580
Visualizing block 18/580
Visualizing block 19/580
Visualizing block 20/580
Visualizing block 21/580
Visualizing block 22/580
Visualizing block 23/580
Visualizing block 24/580
Visualizing block 25/580
Visualizing block 26/580
Visualizing block 27/580
Visualizing block 28/580
Visualizing block 29/580
Visualizing block 30/580
Visualizing block 31/580
Visualizing block 32/580
Visualizing block 33/580
Visualizing block 34/580
Visualizing block 35/580
Visualizing block 36/580
Visualizing block 37/580
Visualizing block 38/580
Visualizing block 39/580
Visualizing block 40/580
Visualizi


KeyboardInterrupt



In [6]:
import open3d as o3d
import numpy as np

# Assuming 'blocks' and 'neighbors' are available and each block has XYZ and potentially RGB information
def visualize_block_and_neighbors(block, neighbors):
    # Create Open3D PointCloud object for the block
    point_cloud_block = o3d.geometry.PointCloud()
    print("ahoj")
    # Extract XYZ coordinates for the block
    xyz_block = block[:, :3]

    # Set points for the block point cloud
    point_cloud_block.points = o3d.utility.Vector3dVector(xyz_block)

    # Set block color to red (same color for all points)
    point_cloud_block.paint_uniform_color([1.0, 0.0, 0.0])

    # Create Open3D PointCloud object for the neighbors
    point_cloud_neighbors = o3d.geometry.PointCloud()

    # Extract XYZ coordinates for the neighbors
    xyz_neighbors = neighbors[:, :3]

    # Set points for the neighbors point cloud
    point_cloud_neighbors.points = o3d.utility.Vector3dVector(xyz_neighbors)

    # Set neighbors color to blue (same color for all points)
    point_cloud_neighbors.paint_uniform_color([0.0, 0.0, 1.0])

    # Visualize both the block and its neighbors
    o3d.visualization.draw_geometries([point_cloud_block, point_cloud_neighbors])

# Visualize the first block of points and its neighbors
visualize_block_and_neighbors(blocks[1], neighbors[1])


In [30]:
import h5py

# Replace with the path to your .h5 file
hdf5_path = r"C:\Users\lukas\Desktop\pointcloud_data.h5"

with h5py.File(hdf5_path, "r") as f:
    # List all datasets in the file
    print("Datasets in the file:")
    for name in f.keys():
        print(name, f[name].shape)

    # Accessing a specific dataset, e.g., 'blocks'
    blocks = f['blocks'][:]
    print("\nFirst block of points:\n", blocks[1])

    # Accessing labels and neighbors information similarly
    labels = f['labels'][:]
    neighbors = f['neighbors'][:]
    intensity = f['intensity'][:]
    
    print("\nFirst block of labels:\n", labels[1])
    print("\nFirst block of neighbor indices:\n", neighbors[1])
    print("\nFirst block of intensity:\n", intensity[1])

Datasets in the file:
blocks (76527, 1024, 3)
intensity (76527, 1024)
labels (0,)
rgb (76527, 1024, 3)

First block of points:
 [[-0.5095989  0.8547    -0.0416   ]
 [-0.502399   0.8715    -0.043499 ]
 [-0.508      0.9169989 -0.05     ]
 ...
 [-2.0165989  3.4196989 -0.1049989]
 [-1.925199   3.3884    -0.102499 ]
 [-1.853999   3.3903    -0.1018989]]


KeyError: "Unable to synchronously open object (object 'neighbors' doesn't exist)"

In [29]:
import os
import laspy
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
import h5py
from sklearn.neighbors import NearestNeighbors

class PointCloudLoader:
    def __init__(self, file_path, block_size=1024, overlap_ratio=0.1, neighbors_k=10):
        """
        Initializes the PointCloudLoader with the provided file path and parameters for block creation.
        
        :param file_path: Path to the point cloud file (e.g., .las).
        :param block_size: Number of points in each block.
        :param overlap_ratio: Ratio of overlap between consecutive blocks.
        :param neighbors_k: Number of neighbors to consider for each point.
        """
        self.file_path = file_path
        self.point_cloud = None
        self.block_size = block_size
        self.overlap_ratio = overlap_ratio
        self.neighbors_k = neighbors_k

    def load_point_cloud(self):
        """
        Loads the point cloud data from the specified file path.
        
        :return: Loaded point cloud as a numpy array.
        """
        file_extension = os.path.splitext(self.file_path)[-1].lower()

        if file_extension == ".las":
            self.point_cloud = self._load_las()
        else:
            raise ValueError(f"Unsupported file format: {file_extension}")

        return self.point_cloud

    def _load_las(self):
        """
        Loads point cloud data from a LAS file, including intensity, RGB, and classification attributes.
        
        :return: Loaded point cloud as a numpy array.
        """
        with laspy.open(self.file_path) as las_file:
            las = las_file.read()
            points = np.vstack((las.x, las.y, las.z)).transpose()

            # Adding additional attributes if they exist
            if hasattr(las, 'intensity'):
                intensity = las.intensity[:, np.newaxis]
                points = np.hstack((points, intensity))

            if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
                rgb = np.vstack((las.red, las.green, las.blue)).transpose() / 65535.0
                points = np.hstack((points, rgb))

            if hasattr(las, 'classification_trees'):
                classification = las.classification_trees[:, np.newaxis]
                points = np.hstack((points, classification))

        return points

    def create_blocks_with_overlap(self):
        """
        Divides the point cloud into overlapping blocks of specified size and overlap ratio.
        
        :return: Blocks of points and corresponding labels.
        """
        points = self.point_cloud[:, :3]
        intensity = self.point_cloud[:, 3] if self.point_cloud.shape[1] > 3 else None
        rgb = self.point_cloud[:, 4:7] if self.point_cloud.shape[1] > 6 else None
        labels = self.point_cloud[:, -1]  if self.point_cloud.shape[1] > 7 else None
        print(labels)
        
        stride = int(self.block_size * (1 - self.overlap_ratio))
        blocks = []
        block_labels = []
        block_intensity = []
        block_rgb = []

        num_points = len(points)
        for start in range(0, num_points, stride):
            end = start + self.block_size
            block_points = points[start:end]
            if intensity is not None:
                block_intensity_points = intensity[start:end]
            if rgb is not None:
                block_rgb_points = rgb[start:end]
            if labels is not None:
                block_labels_points = labels[start:end]

            if len(block_points) == self.block_size:  # Only include full blocks
                blocks.append(block_points)
                if intensity is not None:
                    block_intensity.append(block_intensity_points)
                if rgb is not None:
                    block_rgb.append(block_rgb_points)
                if labels is not None:
                    block_labels.append(block_labels_points)

        return np.array(blocks), np.array(block_labels), np.array(block_intensity), np.array(block_rgb)

    def calculate_neighbors(self, blocks):
        """
        Calculates nearest neighbors for each point in each block.
        
        :param blocks: Array of point blocks.
        :return: Neighbor indices for each block.
        """
        neighbors_info = []

        for block in blocks:
            nbrs = NearestNeighbors(n_neighbors=self.neighbors_k).fit(block)
            _, indices = nbrs.kneighbors(block)
            neighbors_info.append(indices)

        return np.array(neighbors_info)

    def save_to_hdf5(self, hdf5_path, blocks, block_labels, block_intensity, block_rgb, neighbors_info):
        """
        Saves blocks, labels, intensity, RGB, and neighbor information to an HDF5 file.
        
        :param hdf5_path: Path to save the HDF5 file.
        :param blocks: Array of blocks with point data.
        :param block_labels: Labels for points in each block.
        :param block_intensity: Intensity values for points in each block.
        :param block_rgb: RGB values for points in each block.
        :param neighbors_info: Neighbor indices for each block.
        """
        with h5py.File(hdf5_path, "w") as f:
            f.create_dataset("blocks", data=blocks)
            f.create_dataset("labels", data=block_labels)
            f.create_dataset("intensity", data=block_intensity)
            f.create_dataset("rgb", data=block_rgb)
            #f.create_dataset("neighbors", data=neighbors_info)

    def process_and_save(self, hdf5_path):
        """
        Processes the point cloud into blocks, calculates neighbors, and saves to HDF5.
        
        :param hdf5_path: Path to save the HDF5 file.
        """
        blocks, block_labels, block_intensity, block_rgb = self.create_blocks_with_overlap()
        neighbors_info = self.calculate_neighbors(blocks)
        self.save_to_hdf5(hdf5_path, blocks, block_labels, block_intensity, block_rgb, neighbors_info)

# Example usage
if __name__ == "__main__":
    loader = PointCloudLoader(r"C:\Users\lukas\Desktop\stromymale.las", block_size=1024, overlap_ratio=0.0, neighbors_k=10)
    points = loader.load_point_cloud()
    print(points)

    # Process and save to HDF5
    hdf5_path = r"C:\Users\lukas\Desktop\pointcloud_data.h5"
    loader.process_and_save(hdf5_path)


[[-0.5160989   0.6352     -0.0781     ...  0.          0.
   0.        ]
 [-0.5006989   0.6319     -0.0754     ...  0.          0.
   0.        ]
 [-0.49        0.6347     -0.0745     ...  0.          0.
   0.        ]
 ...
 [-0.8228989  -0.7578989  -0.1405989  ...  0.51172656  0.42188144
   0.30469215]
 [-0.7646989  -0.7398989  -0.1532     ...  0.          0.
   0.        ]
 [-0.7333989  -0.7215989  -0.140799   ...  0.41406882  0.34375525
   0.2031281 ]]
None
